<a href="https://colab.research.google.com/github/Dyaruro1/AI-Proyect-PlacementPrediction/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Challenge

##Generalidades

In [1]:
#@title Imports
import gdown
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
#@title Funciones
def load_from_drive(file_id, name):
  gdown.download(id=file_id, output=name, quiet=False)
  return name

def zero_pad(x, target_shape):
    padded = np.zeros(target_shape, dtype=x.dtype)
    slices = tuple(slice(0, min(dim, target)) for dim, target in zip(x.shape, target_shape))
    padded[slices] = x[slices]
    return padded

In [ ]:
#@title Carga de los datos
load_from_drive('1kVbztu0a5E3qefYhMNi7KUgzQXo12uOn', 'data.zip')
!unzip "/content/data.zip" -d "/content/data"

##Exploracion de los datos

In [4]:
df_train = pd.read_csv('/content/data/train.csv')
df_test = pd.read_csv('/content/data/test.csv')

In [5]:
df_train.head()

,id,label
0,sample697.npy,7
1,sample54.npy,81
2,sample2270.npy,4
3,sample1401.npy,99
4,sample1901.npy,43


In [6]:
df_test.head()

,id
0,sample1957.npy
1,sample1369.npy
2,sample782.npy
3,sample2255.npy
4,sample2160.npy


In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2177 entries, 0 to 2176
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2177 non-null   object
 1   label   2177 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 34.1+ KB


In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      545 non-null    object
dtypes: object(1)
memory usage: 4.4+ KB


In [9]:
print(len(df_train['label'].unique()))

101


In [10]:
dir_csv = '/content/data/train.csv'
dir_npys = '/content/data/ot/ot'

chunk_size = 32
EXPECTED_SHAPE = (128,128,125)

In [216]:
all_data = []
all_labels = []

for idx, fila in df_train.iterrows():
    npy_path = os.path.join(dir_npys, fila['id'])
    try:
        npy = np.load(npy_path)
    except:
        print(f"Error cargando {npy_path}")
        continue

    # Verificamos si la imagen es completamente negra o si mas de la mitad son 0
    if np.all(npy == 0) or np.mean(npy == 0) > 0.5:
        continue

    if npy.shape != EXPECTED_SHAPE:  # Espera (128, 128, 125)
        continue

    # Movemos el eje de canales al primero para iterar fácilmente
    npy = np.moveaxis(npy, -1, 0)  # Ahora shape: (125, 128, 128)

    canales_filtrados = []
    for canal in npy:
        # Verificamos que el canal tenga shape (128, 128)
        if canal.shape != (128, 128):
            continue

        proporcion_ceros = np.sum(canal == 0) / canal.size
        if proporcion_ceros < 0.9:
            canales_filtrados.append(canal)

    if len(canales_filtrados) == 0:
        continue

    npy_filtrado = np.array(canales_filtrados)  # shape (canales_filtrados, 128, 128)

    # Volvemos a mover los canales al último eje para conservar el formato original
    npy_filtrado = np.moveaxis(npy_filtrado, 0, -1)  # shape (128, 128, canales_filtrados)

    all_data.append(npy_filtrado)
    all_labels.append(fila['label'])


Error cargando /content/data/ot/ot/sample2451.npy


In [ ]:
# def chunk_generator(dir_csv, dir_npys, chunk_size=32, expected_shape=(128,128,125)):
#     df = pd.read_csv(dir_csv)
#     npys = []
#     labels = []

#     for idx, fila in df.iterrows():
#         npy_path = os.path.join(dir_npys, fila['id'])

#         try:
#             npy = np.load(npy_path)

#             if npy.shape != expected_shape:
#                 try:
#                     npy = zero_pad(npy, expected_shape)
#                 except Exception as e:
#                     print(f"Error haciendo padding: {npy.shape} en {e}")
#                     continue

#             npys.append(npy)
#             labels.append(fila['label'])

#             if len(npys) == chunk_size:
#                 X_chunk = np.array(npys)
#                 y_chunk = np.array(labels)
#                 yield X_chunk, y_chunk
#                 npys = []
#                 labels = []

#         except Exception as e:
#             print(f"Error cargando {npy_path}: {e}")

In [ ]:

# def extract_patches(X, y, patch_size=5):
#     patches = []
#     labels = []
#     margin = patch_size // 2

#     for i in range(X.shape[0]):
#         cube = X[i]
#         label = y[i]
#         for row in range(margin, cube.shape[0] - margin):
#             for col in range(margin, cube.shape[1] - margin):
#                 patch = cube[row-margin:row+margin+1, col-margin:col+margin+1, :]
#                 patches.append(patch)
#                 labels.append(label)

#     patches = np.array(patches)
#     labels = np.array(labels)
#     return patches, labels

In [ ]:
# ### Imports
# # === IMPORTS ===
# import numpy as np
# import pandas as pd
# import os
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
# from tensorflow.keras.optimizers import Adam


# ### Modelo Autoencoder
# input_shape = (5, 5, 125)
# flatten_dim = np.prod(input_shape)
# latent_dim = 64

# input_img = Input(shape=input_shape)
# x = Flatten()(input_img)
# encoded = Dense(256, activation='relu')(x)
# encoded = Dense(latent_dim, activation='relu')(encoded)
# decoded = Dense(256, activation='relu')(encoded)
# decoded = Dense(flatten_dim, activation='sigmoid')(decoded)
# decoded_reshaped = Reshape(input_shape)(decoded)

# autoencoder = Model(input_img, decoded_reshaped)
# encoder = Model(input_img, encoded)
# autoencoder.compile(optimizer=Adam(1e-3), loss='mse')

# ### Entrenamiento
# all_encoded = []
# all_labels = []

# for epoch in range(3):
#     print(f"\n🔁 Epoch {epoch+1}/10")
#     for X_chunk, y_chunk in chunk_generator(dir_csv, dir_npys):
#         X_patches, y_labels = extract_patches(X_chunk, y_chunk)
#         X_patches = X_patches / np.max(X_patches)

#         autoencoder.fit(X_patches, X_patches, epochs=1, verbose=0)

#         encoded = encoder.predict(X_patches)
#         all_encoded.append(encoded)
#         all_labels.extend(y_labels)

# X_encoded = np.vstack(all_encoded)
# patch_labels = np.array(all_labels)

# # Clasificador
# le = LabelEncoder()
# y_encoded = le.fit_transform(patch_labels)
# y_onehot = to_categorical(y_encoded)

# input_latent = Input(shape=(latent_dim,))
# x = Dense(64, activation='relu')(input_latent)
# x = Dense(32, activation='relu')(x)
# output = Dense(y_onehot.shape[1], activation='softmax')(x)

# classifier = Model(input_latent, output)
# classifier.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

# classifier.fit(X_encoded, y_onehot, epochs=10, batch_size=64, shuffle=True, validation_split=0.2)



🔁 Epoch 1/10


Primer entrenamiento con metricas

In [ ]:
!pip install tensorflow

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Flatten, Dense, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Asumiendo que tienes:
# all_data: lista o array numpy (N, 128, 128, 125)
# all_labels: lista o array (N, )

# --- 1. Prepara los datos ---
X = np.array(all_data, dtype=np.float32)
y = np.array(all_labels)

# Normalizar a [0,1]
X = X / np.max(X)

# División train/val
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# --- 2. Define autoencoder convolucional ---

input_shape = (128, 128, 125)
latent_dim = 64

input_img = Input(shape=input_shape)

# Encoder
x = Conv2D(32, (3,3), activation='relu', padding='same')(input_img)
x = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(x)  # 64x64
x = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(x) # 32x32
x = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(x) # 16x16
x = Flatten()(x)
encoded = Dense(latent_dim, activation='relu')(x)

# Decoder
x = Dense(16*16*256, activation='relu')(encoded)
x = Reshape((16, 16, 256))(x)
x = Conv2DTranspose(128, (3,3), activation='relu', padding='same', strides=2)(x)  # 32x32
x = Conv2DTranspose(64, (3,3), activation='relu', padding='same', strides=2)(x)   # 64x64
x = Conv2DTranspose(32, (3,3), activation='relu', padding='same', strides=2)(x)   # 128x128
decoded = Conv2DTranspose(input_shape[-1], (3,3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
encoder = Model(input_img, encoded)

autoencoder.compile(optimizer=Adam(1e-3), loss='mse')

# --- 3. Entrena autoencoder ---
autoencoder.fit(
    X_train, X_train,
    validation_data=(X_val, X_val),
    epochs=20,
    batch_size=8,
    shuffle=True
)

# --- 4. Obtén representación latente ---
X_train_encoded = encoder.predict(X_train)
X_val_encoded = encoder.predict(X_val)

# --- 5. Clasificador ---
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc = le.transform(y_val)

y_train_cat = to_categorical(y_train_enc)
y_val_cat = to_categorical(y_val_enc)

input_latent = Input(shape=(latent_dim,))
x = Dense(64, activation='relu')(input_latent)
x = Dense(32, activation='relu')(x)
output = Dense(y_train_cat.shape[1], activation='softmax')(x)

classifier = Model(input_latent, output)
classifier.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

classifier.fit(
    X_train_encoded, y_train_cat,
    validation_data=(X_val_encoded, y_val_cat),
    epochs=20,
    batch_size=16,
    shuffle=True
)

# --- 6. Evaluación ---
y_val_pred_prob = classifier.predict(X_val_encoded)
y_val_pred = np.argmax(y_val_pred_prob, axis=1)

print("Accuracy en validación:", accuracy_score(y_val_enc, y_val_pred))
print("Reporte de clasificación:\n", classification_report(y_val_enc, y_val_pred, target_names=le.classes_))


Epoch 1/20
194/194 ━━━━━━━━━━━━━━━━━━━━ 510s 3s/step - loss: 0.0627 - val_loss: 0.0080
Epoch 2/20
194/194 ━━━━━━━━━━━━━━━━━━━━ 501s 3s/step - loss: 0.0076 - val_loss: 0.0080
Epoch 3/20
194/194 ━━━━━━━━━━━━━━━━━━━━ 500s 3s/step - loss: 0.0074 - val_loss: 0.0080
Epoch 4/20
194/194 ━━━━━━━━━━━━━━━━━━━━ 498s 3s/step - loss: 0.0076 - val_loss: 0.0080
Epoch 5/20
194/194 ━━━━━━━━━━━━━━━━━━━━ 499s 3s/step - loss: 0.0075 - val_loss: 0.0080
Epoch 6/20
143/194 ━━━━━━━━━━━━━━━━━━━━ 1:57 2s/step - loss: 0.0075

# IMPORTANTE NO CORRER: ES UN MODELO ALTERNATIVO

Preparacion de los datos

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Convertir a numpy array
X = np.array(all_data, dtype=np.float32)  # Shape (N, 128,128,125)
y = np.array(all_labels)

# Normalizar imágenes (por canal o global)
X /= X.max()  # Simple normalización 0-1 (puedes mejorar aquí)

# Codificar etiquetas
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_onehot = to_categorical(y_encoded)


AutoEnconder Convolucional

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Reshape
from tensorflow.keras.models import Model

input_shape = (128, 128, 125)  # Altura, Ancho, Canales
latent_dim = 256

input_img = Input(shape=input_shape)

# Encoder
x = Conv2D(64, (3,3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2,2), padding='same')(x)  # 64x64
x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2,2), padding='same')(x)  # 32x32
x = Conv2D(256, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2,2), padding='same')(x)  # 16x16

x = Flatten()(x)
encoded = Dense(latent_dim, activation='relu')(x)

# Decoder
x = Dense(16*16*256, activation='relu')(encoded)
x = Reshape((16,16,256))(x)

x = UpSampling2D((2,2))(x)  # 32x32
x = Conv2D(256, (3,3), activation='relu', padding='same')(x)
x = UpSampling2D((2,2))(x)  # 64x64
x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
x = UpSampling2D((2,2))(x)  # 128x128
x = Conv2D(64, (3,3), activation='relu', padding='same')(x)

decoded = Conv2D(125, (3,3), activation='sigmoid', padding='same')(x)  # Reconstruir todos los canales

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Encoder model para extracción de características latentes
encoder_model = Model(input_img, encoded)


Entrenamiento del modelo

In [ ]:
autoencoder.fit(X, X, epochs=30, batch_size=16, validation_split=0.2)


Clasificacion usando la representacion latente

In [ ]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model

X_encoded = encoder_model.predict(X)

input_latent = Input(shape=(latent_dim,))
x = Dense(128, activation='relu')(input_latent)
x = Dense(64, activation='relu')(x)
output = Dense(y_onehot.shape[1], activation='softmax')(x)

classifier = Model(input_latent, output)
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

classifier.fit(X_encoded, y_onehot, epochs=20, batch_size=16, validation_split=0.2)
